In [1]:
'''
Yoali Sotomayor Baqueiro  A01650536
David Radames Gómez Chiu  A01334574
Dafne Lecona Cisneros     A01334365
'''

'\nYoali Sotomayor Baqueiro  A01650536\nDavid Radames Gómez Chiu  A01334574\nDafne Lecona Cisneros     A01334365\n'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.utils import to_categorical
import pickle
import keras.models
#import skimage.io
import random
import cv2
#from tqmd import tqmd
DIRECTORY = "img"
FRUITS = ["bananas"]
CATEGORIES = ["good", "black","green"]
IMG_SIZE = 72

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Falta cargar/crear una base de datos. sólo copié lo del profe

# Cargar las imágenes

In [3]:
images = []
for fruit in FRUITS:
    for cat in CATEGORIES:
        path = os.path.join(DIRECTORY,fruit,cat)
        class_num = CATEGORIES.index(cat) #good = 0  | bad = 1, black = 1 | green = 2
        for img in os.listdir(path):
            try:
                
                img_array = cv2.imread(os.path.join(path,img))
                b,g,r = cv2.split(img_array)
                new_array = cv2.merge([r,g,b])
                new_array = cv2.resize(new_array, (IMG_SIZE, IMG_SIZE))
                #plt.subplot(122)
                #plt.imshow(new_array)
                images.append([new_array, class_num])
                
                #plt.imshow(new_array, cmap='gray')  # Graficar un imagen
                #plt.show()  # Mostrar

            except Exception as e:
                print(str(e))
                pass

not enough values to unpack (expected 3, got 0)
not enough values to unpack (expected 3, got 0)
not enough values to unpack (expected 3, got 0)


# Imágenes a matrices

In [4]:
X = []
y = []
random.shuffle(images)
for i, clase in images:
    X.append(i)
    y.append(clase)
    
salida = open("X.pickel","wb")
pickle.dump(X, salida)
salida.close()
salida = open("y.pickel","wb")
pickle.dump(y, salida)
salida.close()

# Modelo

In [5]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam

In [6]:
X = np.array(X)
X = X/X.max() ##NO CORRER MÁS DE UNA VEZ
X = X.reshape(-1,72,72,3)
y = np.array(y)

modelo = Sequential()

modelo.add(Conv2D(32,(3,3),padding="valid",input_shape=X.shape[1:]))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(3,3)))
modelo.add(Conv2D(32,(3,3),input_shape=X.shape[1:]))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Flatten())
modelo.add(Dense(10,activation='relu'))
modelo.add(Dense(15,activation='relu'))
modelo.add(Dense(10,activation='relu'))
modelo.add(Dense(3,activation='relu'))
modelo.add(Dense(3,activation='softmax'))

In [7]:
modelo = Sequential()

modelo.add(Conv2D(32,(7,7),input_shape=X.shape[1:]))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Conv2D(64,(5,5),input_shape=X.shape[1:]))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Conv2D(128,(3,3),input_shape=X.shape[1:]))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Flatten())
modelo.add(Dense(128,activation='relu'))
modelo.add(Dense(3,activation='softmax'))

In [8]:
modelo.compile(optimizer = 'SGD', loss = 'categorical_crossentropy',metrics=['categorical_accuracy'])
X2 = np.array(X,dtype='float32')
modelo.fit(X2,to_categorical(y),batch_size=32, epochs=30, validation_split=.25) 

Train on 1785 samples, validate on 595 samples
Epoch 1/30
1785/1785 [==============================] - 20s 11ms/step - loss: 0.9823 - categorical_accuracy: 0.5154 - val_loss: 0.7828 - val_categorical_accuracy: 0.7950
Epoch 2/30
1785/1785 [==============================] - 20s 11ms/step - loss: 0.6744 - categorical_accuracy: 0.7333 - val_loss: 0.3921 - val_categorical_accuracy: 0.7950
Epoch 3/30
1785/1785 [==============================] - 20s 11ms/step - loss: 0.6411 - categorical_accuracy: 0.8151 - val_loss: 0.3292 - val_categorical_accuracy: 0.8454
Epoch 4/30
1785/1785 [==============================] - 20s 11ms/step - loss: 0.2573 - categorical_accuracy: 0.8992 - val_loss: 0.1414 - val_categorical_accuracy: 0.9731
Epoch 5/30
1785/1785 [==============================] - 19s 11ms/step - loss: 0.2406 - categorical_accuracy: 0.9188 - val_loss: 0.1666 - val_categorical_accuracy: 0.9311
Epoch 6/30
1785/1785 [==============================] - 19s 11ms/step - loss: 0.1193 - categorical_accu

In [9]:
def carga(rutaImg):
    imgSize = 72
    imgArray = cv2.imread(os.path.join(path,img))
    b,g,r = cv2.split(img_array)
    nImgArray = cv2.merge([r,g,b])
    nImgArray = cv2.resize(new_array, (IMG_SIZE, IMG_SIZE))
    return nImgArray.reshape(-1, imgSize, imgSize, 3)

In [12]:
modelo.save("modelo.model")
modelo = keras.models.load_model('modelo.model')
nombre = input("imagen? ")
p = modelo.predict([carga(nombre)])
print("p:",p)
indice = np.argmax(p[0])
print(CATEGORIES[indice])

imagen? verde.jpeg
p: [[0. 0. 1.]]
green
